# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 10:46AM,240837,10,MSP211766,"Methapharm, Inc.",Released
1,Jun 30 2022 10:46AM,240837,10,MSP211767,"Methapharm, Inc.",Released
2,Jun 30 2022 10:46AM,240837,10,MSP211768,"Methapharm, Inc.",Released
3,Jun 30 2022 10:46AM,240837,10,MSP211769,"Methapharm, Inc.",Released
4,Jun 30 2022 10:46AM,240837,10,MSP211770,"Methapharm, Inc.",Released
5,Jun 30 2022 10:46AM,240837,10,MSP211771,"Methapharm, Inc.",Released
6,Jun 30 2022 10:46AM,240837,10,MSP211772,"Methapharm, Inc.",Released
7,Jun 30 2022 10:46AM,240837,10,MSP211776,"Methapharm, Inc.",Released
8,Jun 30 2022 10:46AM,240837,10,MSP211777,"Methapharm, Inc.",Released
9,Jun 30 2022 10:46AM,240837,10,MSP211778,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,240833,Released,6
36,240834,Released,1
37,240835,Released,1
38,240836,Released,2
39,240837,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240833,NaN,6.0
240834,NaN,1.0
240835,NaN,1.0
240836,NaN,2.0
240837,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240750,25.0,1.0
240766,0.0,1.0
240768,0.0,1.0
240774,1.0,0.0
240776,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240750,25,1
240766,0,1
240768,0,1
240774,1,0
240776,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,0,1
2,240768,0,1
3,240774,1,0
4,240776,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,,1
2,240768,,1
3,240774,1,
4,240776,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc."
10,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc."
12,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation
13,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation
19,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc."
20,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation
57,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC"
58,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.
59,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.
60,Jun 30 2022 9:59AM,240828,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc.",,10
1,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",,2
2,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,,1
3,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,,6
4,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",,1
5,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,,37
6,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",,1
7,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,,1
8,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,,1
9,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc.",10,
1,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",2,
2,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1,
3,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,
4,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1,
5,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,37,
6,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1,
7,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1,
8,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1,
9,Jun 30 2022 9:59AM,240828,19,ACG North America LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc.",10,
1,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",2,
2,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1,
3,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,
4,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc.",10.0,NaN
1,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",2.0,NaN
2,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1.0,NaN
3,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,NaN
4,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:46AM,240837,10,"Methapharm, Inc.",10.0,0.0
1,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",2.0,0.0
2,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1.0,0.0
3,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,0.0
4,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3130655,178.0,45.0
12,481619,1.0,2.0
16,722424,5.0,4.0
17,240768,1.0,0.0
18,240813,1.0,0.0
19,963263,35.0,24.0
20,1685581,25.0,50.0
21,1203910,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3130655,178.0,45.0
1,12,481619,1.0,2.0
2,16,722424,5.0,4.0
3,17,240768,1.0,0.0
4,18,240813,1.0,0.0
5,19,963263,35.0,24.0
6,20,1685581,25.0,50.0
7,21,1203910,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,178.0,45.0
1,12,1.0,2.0
2,16,5.0,4.0
3,17,1.0,0.0
4,18,1.0,0.0
5,19,35.0,24.0
6,20,25.0,50.0
7,21,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,178.0
1,12,Released,1.0
2,16,Released,5.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Executing,45.0,2.0,4.0,0.0,0.0,24.0,50.0,2.0
Released,178.0,1.0,5.0,1.0,1.0,35.0,25.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Executing,45.0,2.0,4.0,0.0,0.0,24.0,50.0,2.0
1,Released,178.0,1.0,5.0,1.0,1.0,35.0,25.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Executing,45.0,2.0,4.0,0.0,0.0,24.0,50.0,2.0
1,Released,178.0,1.0,5.0,1.0,1.0,35.0,25.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()